In [1]:
import pandas as pd 
import json

In [10]:
! mv /Users/spangher/Downloads/combined_test_prompt1_v2.json ../data/v2_queries/

In [11]:
source_df_with_clusters = pd.read_csv('cache/2024-09-19__source-df-with-all-clusters.csv', index_col=0)

In [13]:
test_set_sources = json.load(open('../data/v2_queries/combined_test_prompt1_v2.json'))

In [14]:
len(test_set_sources)

11628

In [19]:
test_set_sources_df = pd.DataFrame(test_set_sources).explode('sources').pipe(lambda df: pd.concat([
    df[['url', 'query']].reset_index(drop=True),
    pd.DataFrame(df['sources'].tolist())
], axis=1))

In [52]:
test_set_df_with_oracle_sources = (
    test_set_sources_df
     # .explode('Original Name')
 .merge(
     # source_df_with_clusters.drop(columns='Name'),#.explode('Original Name'), 
     # left_on=['url', 'Original Name'], 
     # right_on=['url', 'Original Name'],
     source_df_with_clusters.drop(columns='Original Name'),#.explode('Original Name'), 
     left_on=['url', 'Name'], 
     right_on=['url', 'Name'],
     how='left'
 )
 .assign(matched=lambda df: df['Narrative Function'].notnull())
)

In [159]:
labels_to_use = source_df_with_clusters['cluster_descript_leaf_3'].str.split(':').str.get(0).value_counts()
inferred_to_human_label_mapper = {
    '"Insightful Resource"': "Expert Insights",                  
    '"Analytical Depth"': "Analysis and Criticism",    
    '"Credibility and Engagement"': "Credibility and Engagement",                     
    '"Influential Source"': "Central Figure",            
    '"Alternative Viewpoints"': "Alternative Viewpoints",    
    '"Contextual Insight"': "Background Information",    
    '"Contextual Information"': "Background Information",    
    '"Contextual Insights"': "Peripheral Context",        
    '"Key Source"': "Central Figure",
    '"Extensive Expert Analysis"': "Expert Analysis",
    '"Contextual Guidance"': "Author Perspective",        
    '"Practical Case Studies"': "Anecdotes",                 
    '"Background Insight Providers"': "Background Information",    
    '"Background Enrichment Sources"': "Examples and Illustration", 
    '"Credibility Anchor"': "Authoritative Source",     
}

In [169]:
test_set_df_with_oracles_mapped = (
    test_set_df_with_oracle_sources
         .assign(oracle_label=lambda df: 
                 df['cluster_descript_leaf_3'].str.split(':').str.get(0).map(inferred_to_human_label_mapper)
                )
            [['url', 'query', 'Name', 'Original Name', 'Information', 'oracle_label', 'matched']]
)
output_test_df_w_oracles = (
    test_set_df_with_oracles_mapped
         .groupby(['url', 'query'])
         .apply(lambda df: pd.Series({
             'sources': df[['Name', 'Original Name', 'Information', 'oracle_label']].to_dict(orient='records'),
             '% match': df['matched'].mean()
         }))
)


/var/folders/xh/qnyq7yzj0r328_7hnb7pgxth0000gp/T/ipykernel_7229/1829112242.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: pd.Series({


In [161]:
output_matched_test_df_w_oracles = (
    output_test_df_w_oracles
         .loc[lambda df: df['% match'] > .85]
         .reset_index()
)

In [162]:
output_matched_test_df_w_oracles['sources'].explode().str.get('oracle_label').value_counts()

sources
Expert Insights               4149
Background Information        3525
Analysis and Criticism        3155
Central Figure                2991
Credibility and Engagement    2384
Alternative Viewpoints        1930
Peripheral Context            1114
Expert Analysis               1075
Author Perspective             939
Anecdotes                      865
Examples and Illustration      716
Authoritative Source           477
Name: count, dtype: int64

In [163]:
(output_matched_test_df_w_oracles
 .drop(columns='% match')
 .to_json('../data/v2_queries/combined_test_prompt1_v2__with_oracle.json', orient='records')
)

In [142]:
# ! open ../data/v2_queries

In [158]:
import pyperclip
pyperclip.copy('\n\n'.join(source_df_with_clusters[['cluster_descript_leaf_3']]
 .drop_duplicates()
 .assign(label=lambda df: df['cluster_descript_leaf_3'].str.split(':').str.get(0))
 .assign(human_label=lambda df: df['label'].map(inferred_to_human_label_mapper))
 .assign(cluster_descript_leaf_3=lambda df: df.apply(lambda x: x['cluster_descript_leaf_3'].replace(x['label'], '"{}"'.format(x['human_label'])), axis=1))
 ['cluster_descript_leaf_3'].tolist()
               ))

"Credibility and Engagement": These sources contribute to building trust, influence audience understanding, and enhancing the reliability of information by providing personal accounts, insights and warnings.

"Authoritative Source": These sources provide verified, expert-backed information that enhances the trustworthiness and reliability of the content they support.

"Background Information": These sources provide broader context to events, helping readers understand the main topic in the context of what is going on and grasp peripheral details.

"Analysis and Criticism": These sources are experts and insiders. They offer deep insights, critiques, and multifaceted interpretations of complex issues and decisions across diverse fields.

"Central Figure": These sources are the main individual, company, or entity featured in news articles, serving as the focal point by providing important information, insights, actions, decisions, and statements.

"Examples and Illustration": These sources provide background information in different ways, enhancing the main content for a more comprehensive grasp of topics discussed.

"Alternative Viewpoints": These sources offer diverse perspectives, critical analyses, and opposing opinions to provide a more balanced understanding and challenge dominant views across various topics.

"Expert Analysis": These sources offer a broad and detailed range of expert analyses and insights, enriching the article's depth and credibility.

"Expert Insights": These sources collectively provide essential data, context, analysis, and credibility across diverse topics, enhancing depth and understanding in journalistic articles and ensuring comprehensive and accurate reporting.

"Author Perspective": These sources that offer analysis and personal insights, usually from the author of the article.

"Anecdotes": These sources offer diverse and specific examples that highlight different responses, adaptations, successes, challenges, innovations, and impacts across sectors, providing insights into broader themes and concepts.

"Peripheral Context": These sources enrich the main content by offering deeper context than typical background sources.

,url,query,Name,Original Name,Information,oracle_label,matched
0,www.thestar.com/news/world/2011/03/10/florida_...,What are the implications of law enforcement a...,Miami-Dade Police Department,Miami-Dade police department,The Miami-Dade police department is planning t...,NaN,False
1,www.thestar.com/news/world/2011/03/10/florida_...,What are the implications of law enforcement a...,Aida Fina-Milian,Aida Fina-Milian,"Aida Fina-Milian, a detective, stated that the...",NaN,False
2,www.thestar.com/news/world/2011/03/10/florida_...,What are the implications of law enforcement a...,Federal Aviation Administration,Federal Aviation Administration,The use of the MAV on city streets has not yet...,NaN,False
3,www.thestar.com/news/world/2011/03/10/florida_...,What are the implications of law enforcement a...,Andrew Cohen,Andrew Cohen,Sergeant Andrew Cohen of the Miami-Dade police...,NaN,False
4,www.thestar.com/news/world/2011/03/10/florida_...,What are the implications of law enforcement a...,Honeywell Aerospace,Honeywell Aerospace,The MAV was manufactured by Honeywell Aerospac...,NaN,False
...,...,...,...,...,...,...,...
98352,www.expressnews.com/news/local/article/Tyson-v...,What is the impact of Tyson Foods' decision to...,meatpacking plants,meatpacking plants,Tyson's mandate is subject to negotiations wit...,NaN,False
98353,www.expressnews.com/news/local/article/Tyson-v...,What is the impact of Tyson Foods' decision to...,Seguin,Seguin,About 60 percent of the city of Seguin's workf...,Expert Insights,True
98354,www.expressnews.com/news/local/article/Tyson-v...,What is the impact of Tyson Foods' decision to...,Green Gate Garden Center,Green Gate Garden Center,Green Gate Garden Center in Seguin began manda...,Analysis and Criticism,True
98355,www.expressnews.com/news/local/article/Tyson-v...,What is the impact of Tyson Foods' decision to...,Robyn Wolters,Robyn Wolters,Office manager Robyn Wolters said that they re...,Analysis and Criticism,True


In [206]:
examples_mapped_df = (
    test_set_df_with_oracles_mapped[['url', 'Name', 'Information']]
     .merge(source_df_with_clusters
             .assign(label=lambda df: df['cluster_descript_leaf_3'].str.split(':').str.get(0))
             .assign(human_label=lambda df: df['label'].map(inferred_to_human_label_mapper))
             [['url', 'Name', 'Narrative Function', 'human_label']]
))

In [208]:
t = (examples_mapped_df
 .groupby('human_label').apply(lambda df: df.head().reset_index(drop=True))
)

/var/folders/xh/qnyq7yzj0r328_7hnb7pgxth0000gp/T/ipykernel_7229/2021855737.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .groupby('human_label').apply(lambda df: df.head().reset_index(drop=True))


In [209]:
cats = list(set(map(lambda x: x[0], t.index)))

In [210]:
cats

['Background Information',
 'Credibility and Engagement',
 'Examples and Illustration',
 'Author Perspective',
 'Expert Analysis',
 'Peripheral Context',
 'Anecdotes',
 'Alternative Viewpoints',
 'Authoritative Source',
 'Expert Insights',
 'Analysis and Criticism',
 'Central Figure']

In [227]:
len(cats)

12

In [246]:
idx = 11
cats[idx]

'Central Figure'

In [249]:
t2 = (t.loc[cats[idx]][['Name', 'Information', 'Narrative Function']]
 .rename(columns={'Name': 'Source name', 'Information': 'Info. provided', 'Narrative Function': 'Narrative function'})
 .iloc[[0, 3]].to_dict(orient='records')
)
# for t_i in t2:
    # for k,v in t_i.items():
        # print(f'{k}: {v}')
    # print()
t2

[{'Source name': 'Sam Bankman-Fried',
  'Info. provided': 'Sam Bankman-Fried is the founder of FTX and former chief executive of FTX. He was arrested in the Bahamas and appeared in court, where he was denied bail. He was taken away in handcuffs and was described as a modern-day John Pierpont Morgan, but is now likened to Bernie Madoff, the fraudster who orchestrated a notorious Ponzi scheme. He was a major donor to the Democratic Party and known for his commitment to effective altruism, a charitable movement that urges adherents to give away their wealth in efficient and logical ways.',
  'Narrative function': '"Main Actor": This source is the main actor in the news article and the subject of the story.'},
 {'Source name': 'Boeing',
  'Info. provided': 'Boeing admitted that it may never know what caused the battery malfunctions that resulted in all its 787 Dreamliner aircraft being grounded. Replacement battery systems are now being fitted to all 50 Dreamliners that had been in operati

'"Expert Insights": 
    {'Source name': 'S&P Global',
    'Info. provided': 'The value of deposites relative to a company's overall worth.',
    'Narrative function': 'In a story about a company's value, this source provides data and statistics about the situation.'},
    {'Source name': 'Irene Knapp',
    'Info. provided': 'Insights about the lack of executive leadership at a company',
    'Narrative function': 'In a story about a company's failings, this source is used to express concerns and criticisms about culture and leadership',}

"Central Figure":
    {'Source name': 'Joyann Ferguson-Pratt',
      'Info. provided': "A judge who makes a key ruling in a case being discussed in the article.",
      'Narrative function': 'In a story about a case, this source made decision that affects the main actor.'},
     {'Source name': 'Larry Loftis',
      'Info. provided': "The general manager company provides his view on where the company is going based on his leadership",
      'Narrative function': "In a story about a company, this source provides detailed information about the company\'s actions, decisions, and technology.'}

"Examples and Illustration":
    {'Source name': 'NASA',
     'Info. provided': "Information about a flight they piloted.",
      'Narrative function': 'In a story about worldwide space programs, provides context and information about US government's priorities, goals, and timelines.'}
    {'Source name': 'A local bank',
     'Info. provided': 'States that all shareholders get the same information at the same time.',
    'Narrative function': 'In a story about the effects of regulations, this source is used to provide context and background information on how a regulation is being followed.'}

'Author Perspective':
    {'Source name': 'Ross Marchand',
     'Info. provided': 'The director of policy at an organization with a general opinion, writes an article.',
      'Narrative function': 'This source is used to provide the author\'s opinion and perspective on the issue.'},
     {'Source name': 'David Ingram',
      'Info. provided': 'David Ingram is a journalist who covers tech for NBC News.',
      'Narrative function': 'The author of the article, providing context and background information.'}

'Expert Analysis':
    {'Source name': 'Martin Sankey',
      'Info. provided': 'A senior research analyst who has been following a company closely.',
      'Narrative function': In a story about a company, this source provides expert opinion from a senior research analyst.'},
     {'Source name': 'Brian Fry',
      'Info. provided': 'Provides advice on investment apps and offers insights on how to choose the best app for individual needs and goals.',
      'Narrative function': 'In a story about investment apps, this source provides expert opinions on investment apps.'}

"Peripheral Context":
    {'Source name': 'HBO',
     'Info. provided': 'Released a documentary about a company, which keeps it in the public eye and affect the job prospects of former employees.',
      'Narrative function': 'In a story about how a company's misfortunes went public, this source provides context for the unfolding events.'},
     {'Source name': 'Attorneys',
      'Info. provided': 'Presented closing arguments in a trial.',
      'Narrative function': 'In a story about a trial, this source is used to provide context about the trial and the actions of the lawyers involved.'}

'Anecdotes':
    {'Source name': 'Japan Airlines',
      'Info. provided': "Infomration about a battery overheating incident.",
      'Narrative function': "In a story about accidents, this source is used as an example of an airline that experienced a battery malfunction incident.'},
     {'Source name': 'Fidelity Go',
      'Info. provided': 'Offers an easy-to-understand investment app.',
      'Narrative function': 'In a story about investment apps, this source provides an example of one.'}

'Alternative Viewpoints':
    {'Source name': 'Airbus',
      'Info. provided': 'Airbus was planning to use lithium-ion batteries in its new plane, the A350, but has decided against it after watching Boeing struggle to prove they are safe.',
      'Narrative function': 'In a story about the decisions of a company, this source is used as a comparison.'},
    {'Source name': 'TD Ameritrade',
  'Info. provided': 'This company offers a range of investment products and services, including brokerage accounts, robo-advisors, and financial planning tools.',
  'Narrative function': 'In a story about investment apps, this source is used to provide a counterpoint to other investment apps.'}

  
'Authoritative Source':
    {'Source name': 'US aircraft regulators',
      'Info. provided': 'US aircraft regulators approved a revamped battery design for the 787.',
      'Narrative function': 'In a story about airplane batteries, this source is an authority about an issue and is involved in the events.'},
     {'Source name': 'Commodity Futures Trading Commission',
      'Info. provided': "Alleges that an actor took money that customers wanted to use.",
      'Narrative function': 'In a story about corporate malfeasance, this source has an inside look involved in the events of the story.'}

"Expert Insights":
{'Source name': 'Sundar Pichai',
'Info. provided': "The source provided insights about a technology being a tremendously positive force.",
  'Narrative function': 'In a story about AI, this source is used to provide a statement from Google\'s CEO.'},
 {'Source name': 'Google',
  'Info. provided': "Provides information about company structure.",
  'Narrative function': 'In a story about a company, this source is used to provide factual information about Google, its policies, and its actions.'}


'Analysis and Criticism':
{'Source name': 'Peter Schoomaker',
  'Info. provided': 'This source provides an insider\'s explanation for the cancellation of a program and its implications.',
  'Narrative function': 'In a story about US Army decision making, this source explains and pushes back on some information.'},
 {'Source name': 'Google employees',
  'Info. provided': "Employees who asked questions about a memo had their comments and identities leaked elsewhere online.",
  'Narrative function': 'In a story about a corporate mishap, this source provides insight into what happened and criticizes the company.'}

"Central Figure":
{'Source name': 'Sam Bankman-Fried',
  'Info. provided': 'The founder of a company.',
  'Narrative function': 'In a story about the company, this source is the main actor in the news article and the subject of the story.'},
 {'Source name': 'Boeing',
  'Info. provided': 'The company admitted that it may never know what caused the battery malfunctions.',
  'Narrative function': 'In a story about a company's malfeasance, this source is the main actor in the news article and provides information about the company\'s actions and decisions.'}


# Old Work

In [ ]:
train_queries = json.load(open('../data/v2_queries/query_train_prompt1_v2.json'))
test_queries = json.load(open('../data/v2_queries/query_test_prompt1_v2.json'))
train_query_df = pd.DataFrame(train_queries)
train_query_df['split'] = 'train'
test_query_df = pd.DataFrame(test_queries)
test_query_df['split'] = 'test'
full_query_df = pd.concat([train_query_df, test_query_df])

test_narr = json.load(open('../data/v2_narr_parsed/v2_test_set_narr.json'))
train_narr = json.load(open('../data/v2_narr_parsed/v2_train_set_narr.json'))

narr_df = pd.concat([
    pd.DataFrame(train_narr),
    pd.DataFrame(test_narr)
])
source_narr_df = narr_df.explode('sources').dropna().reset_index(drop=True)
source_narr_df = pd.concat([
    source_narr_df[['url']],
    source_narr_df['sources'].pipe(lambda s: pd.DataFrame(s.tolist()))
], axis=1)

source_narr_df.iloc[0].to_dict()

In [9]:
(
    source_df_with_clusters
         .loc[lambda df: df['cluster_descript_leaf_1'] == df['cluster_descript_leaf_1'].unique()[1]]
         .drop_duplicates('Narrative Function').iloc[0].to_dict()
)

{'url': 'www.theatlantic.com/sponsored/qualcomm-2016/the-space-within/768/',
 'Name': 'Researchers',
 'Original Name': 'researchers',
 'Narrative Function': '"Authority": This source is used to provide information about the Framingham Heart Study and its findings.',
 'cluster_descript_leaf_1': '"Expert Endorsement": These sources are used to provide authoritative and credible information, data, and guidance on various aspects of the COVID-19 pandemic, including vaccines, testing, public health guidelines, and pandemic control measures.',
 'cluster_descript_leaf_2': '"Authoritative Source": These sources provide credible, expert-backed information and validations that lend legitimacy and reliability across various topics, enhancing the trust and accuracy of the articles they support.',
 'cluster_descript_leaf_3': '"Credibility Anchor": These sources provide verified, expert-backed information that enhances the trustworthiness and reliability of the content they support.',
 'cluster_desc

In [7]:
source_df_with_clusters['cluster_descript_leaf_4'].value_counts()#$.head(2)

cluster_descript_leaf_4
"Core Narrative Sources": These sources provide key insights, credibility, context, and diverse perspectives crucial for developing, substantiating, and advancing the main narrative of the article.                                                                                           148915
"Extensive Context"\n\n"These sources provide in-depth background, integral details, and thorough narrative foundations that enhance the understanding of various topics and events, ensuring comprehensive insight."                                                                            70762
"Insightful Narratives": These sources offer detailed, informative, and contextual explanations with comprehensive data and innovative solutions, enhancing reader understanding and decision-making across various subjects.                                                                    55396
"Nuanced Discourse": These sources offer comparative analysis, opposing viewpoints, and cri

In [124]:
# source_narr_df[['url', 'Name', 'Perspective', 'Centrality']].merge(source_df_with_clusters, on=['url', 'Name'])

In [9]:
source_df_with_clusters_and_queries = source_df_with_clusters.merge(full_query_df)

In [15]:
source_df_with_clusters_and_queries.iloc[0]

url                        blog.cleveland.com/metro/2011/01/light_snow_ca...
Name                                                                  Police
Original Name                                                 Police, police
Narrative Function                                            Primary Source
cluster_descript_leaf_1                                       Primary Source
cluster_descript_leaf_2                                       Primary Source
cluster_descript_leaf_3                             Primary Narrative Source
cluster_descript_leaf_4                               Core Narrative Sources
cluster_descript_leaf_5                                  Credible Narratives
query                      What is the impact of a light snowfall on traf...
split                                                                   test
Name: 0, dtype: object

In [10]:
to_process = ['Narrative Function', 'cluster_descript_leaf_1', 'cluster_descript_leaf_2', 'cluster_descript_leaf_3', 'cluster_descript_leaf_4', 'cluster_descript_leaf_5']
source_df_with_clusters_and_queries[to_process] = (
    source_df_with_clusters_and_queries
     [to_process]
     .apply(lambda s: s.str.split(':').str.get(0).str.split('\n\n').str.get(0).str.strip().str.replace('"', ''))
)

In [11]:
broad_clusters_categories = (
    source_df_with_clusters_and_queries
         .groupby('url')['cluster_descript_leaf_5']
         .value_counts()
         .unstack()
)

In [ ]:
## classification problem:

## input: initial query — initial summary of the story 
## predict: multilabel output of the different source-categories


## process —
##    start with your query
##    planner tells you what KINDS of sources you need
##    execute query 1
##       you determine that you filled "Central Figure" category of source
##       your planner tells you you still need an "Enriched Narratives" category of source
##    this helps you formulate/execute a better followup/interleaving query
## -> can we make an HMM or Graph neural network to predict this?

## directly sample from the distribution of discourse labels?
##    slightly more advanced - use kmeans clustering to cluster 5-6 clusters of different story types and then try to assign 
##    stories to each cluster

In [192]:
train_query_df.iloc[0].to_dict()

{'url': 'www.vice.com/en/article/jg8743/facebook-spending-dollar50m-rese-to-not-ruin-metaverse-like-it-ruined-the-real-world',
 'query': 'Can a company like Facebook, with a history of contributing to real-world problems, be trusted to responsibly develop a virtual world like the metaverse?',
 'split': 'train'}

In [190]:
broad_clusters_categories.sort_values('Enriched Narratives', ascending=False)

cluster_descript_leaf_5,Central Figure,Contextual Narratives,Credible Narratives,Critical Analysis,Enriched Narratives,User Interaction
url,,,,,,
www.fox23.com/news/trending/teacher-appreciation-week-2021-deals-freebies-educators/SOQPCHBO3JAC3MXNVHGY3IFYHM/,NaN,NaN,2.0,NaN,24.0,NaN
www.chron.com/life/pets/article/New-website-guesses-what-dog-breed-you-are-6842859.php,NaN,NaN,NaN,NaN,24.0,NaN
www.usatoday.com/story/travel/flights/todayinthesky/2017/09/11/irma-snarling-flights-five-major-airline-hubs-all-once/653483001/,NaN,NaN,1.0,NaN,22.0,NaN
www.theguardian.com/technology/2014/feb/12/10-things-to-know-about-lifelogging,NaN,NaN,2.0,NaN,21.0,NaN
www.cbsnews.com/miami/news/a-guide-to-veterans-day-2020-deals-and-freebies/,NaN,NaN,1.0,NaN,21.0,NaN
...,...,...,...,...,...,...
wwwrm1.ansa.it/pressrelease/canale_salute_benessere/index.shtml,NaN,NaN,1.0,NaN,NaN,NaN
y108.cbslocal.com/2014/07/23/luke-bryan-to-honor-u-s-troops-with-free-tickets-to-chicago-concert/,1.0,NaN,1.0,NaN,NaN,NaN
youpix.virgula.uol.com.br/app/os-114-aplicativos-mais-uteis-pra-voce-baixar-agora/,1.0,1.0,11.0,1.0,NaN,NaN


In [175]:
training_data = (
    broad_clusters_categories
        .fillna(0)
        .pipe(lambda df: (df > 0).astype(float))
        .merge(full_query_df, right_on='url', left_index=True, how='left')
)
#.pipe(lambda df: df.divide(df.sum(axis=1), axis=0))#.sort_values('Contextual Narratives', ascending=False))

# Train BERT Classifier

In [182]:
training_data.to_csv('../narrative_function/bert_training_data.csv')